## Develop `wltp/cycler.py`
(WIP) run a vehicle from the h5db, step-by-step.

In [1]:
## To autoreload codein python files here.
%load_ext autoreload
%autoreload 2

## Auto-format cells to ease diffs.
%load_ext lab_black

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
## If you change that, restart kernel and clear all outpouts before running it
#%matplotlib widget
%matplotlib inline

In [3]:
from typing import Union, List, Callable, Any, Sequence as Seq
import io
import itertools as itt
import logging
from pathlib import Path, PurePosixPath as P
import sys

from columnize import columnize
import numpy as np
import pandas as pd
from pandas import HDFStore, IndexSlice as idx
from pandas.core.generic import NDFrame
from matplotlib import pyplot as plt
from pandalone.mappings import Pstep
import qgrid

from graphtik import operation, compose, optional, sideffect
import wltp
from wltp.experiment import Experiment
from wltp.utils import yaml_loads
from wltp import datamodel, io as wio, engine, vmax, vehicle, cycler

## Add tests/ into `sys.path` to import `vehdb` module.
#
proj_dir = str(Path(wltp.__file__).parents[1] / "tests")
if proj_dir not in sys.path:
    sys.path.insert(0, proj_dir)

import vehdb

idx = pd.IndexSlice
log = logging.getLogger("VMax.ipynb")
logging.getLogger("blib2to3").propagate = False  # Disable `lab_black` logs.
logging.basicConfig(
    level=logging.DEBUG,
    format="%(asctime)s|%(levelname)5.5s|%(name)s:[%(funcName)s]:\n  +--> %(message)s",
    datefmt="%Y-%m-%d,%H:%M:%S",
)
pd.set_option("display.max_columns", 64)

In [4]:
from oct2py import octave

%load_ext oct2py.ipython

In [13]:
octave.addpath(str(Path("Matlab").absolute()))

'.:/home/ankostis/Work/wltp.git/venv/lib/python3.7/site-packages/oct2py:/home/ankostis/Work/wltp.git/venv/lib/python3.7/site-packages/octave_kernel:/usr/lib/x86_64-linux-gnu/octave/5.1.0/site/oct/x86_64-pc-linux-gnu:/usr/lib/x86_64-linux-gnu/octave/site/oct/api-v53/x86_64-pc-linux-gnu:/usr/lib/x86_64-linux-gnu/octave/site/oct/x86_64-pc-linux-gnu:/usr/share/octave/5.1.0/site/m:/usr/share/octave/site/api-v53/m:/usr/share/octave/site/m:/usr/share/octave/site/m/startup:/usr/lib/x86_64-linux-gnu/octave/5.1.0/oct/x86_64-pc-linux-gnu:/usr/share/octave/5.1.0/m:/usr/share/octave/5.1.0/m/audio:/usr/share/octave/5.1.0/m/debian:/usr/share/octave/5.1.0/m/deprecated:/usr/share/octave/5.1.0/m/elfun:/usr/share/octave/5.1.0/m/general:/usr/share/octave/5.1.0/m/geometry:/usr/share/octave/5.1.0/m/gui:/usr/share/octave/5.1.0/m/help:/usr/share/octave/5.1.0/m/image:/usr/share/octave/5.1.0/m/io:/usr/share/octave/5.1.0/m/java:/usr/share/octave/5.1.0/m/legacy:/usr/share/octave/5.1.0/m/linear-algebra:/usr/share/

In [166]:
!cd Matlab
octave.calc_all_cases()

read input files :
- case.txt
- vehicle.txt
- engine.txt
  ...reading 1 out of 5095 in 46.8 sec
  ...reading 1468 out of 5095 in 53.8 sec
  ...reading 2988 out of 5095 in 60.8 sec
  ...reading 4560 out of 5095 in 67.8 sec
- gearbox.txt
- phase.txt
- trace.txt
  ...reading 1 out of 7015 in 63.5 sec
  ...reading 175 out of 7015 in 70.6 sec
  ...reading 336 out of 7015 in 77.6 sec
  ...reading 493 out of 7015 in 84.6 sec
  ...reading 640 out of 7015 in 91.6 sec
  ...reading 795 out of 7015 in 98.6 sec
  ...reading 947 out of 7015 in 105.7 sec
  ...reading 1092 out of 7015 in 112.7 sec
  ...reading 1229 out of 7015 in 119.7 sec
  ...reading 1372 out of 7015 in 126.8 sec
  ...reading 1519 out of 7015 in 133.8 sec
  ...reading 1666 out of 7015 in 140.8 sec
  ...reading 1813 out of 7015 in 147.8 sec
  ...reading 1960 out of 7015 in 154.9 sec
  ...reading 2106 out of 7015 in 161.9 sec
  ...reading 2252 out of 7015 in 168.9 sec
  ...reading 2398 out of 7015 in 175.9 sec
  ...reading 2544 out of

Oct2PyError: Octave evaluation error:
error: element number 1 undefined in return list